In [ ]:
import torch
x = torch.tensor([
    [ 0.0470, -0.0431,  0.0421],
    [ 0.0282, -0.0119,  0.0402],
    [0.0181, 0.0000, 0.0000],
    [-0.0017,  0.0001, -0.0004],
    [ 4.1447e-03, -2.7173e-03, -2.3842e-07]
    ])
x.softmax(1)

In [ ]:
torch.softmax(-x*20, 1)

In [ ]:
torch.softmax(x.exp()-1, 1)

In [ ]:
torch.log(torch.tensor([1/3, 1, 2/1]))

In [ ]:
x.exp()/2

In [ ]:
x1 = x - x.min(1, keepdim=True)[0] + 1e-3
x1 / x1.sum(1, keepdim=True)

# TOXIC dpo

- turn exach pair into ds row
- shuffle

In [ ]:
from datasets import load_dataset
dataset = load_dataset('unalignment/toxic-dpo-v0.1') # this should give a a bigger # use map in batched mode to return more rows than it got
def transform(row):
    return {
        "chosen": [{'role':'user', 'content': row['prompt']},{'role':'assistant', 'content': row['chosen']}],
        "rejected": [{'role':'user', 'content': row['prompt']},{'role':'assistant', 'content': row['rejected']}]
    }

dataset1 = dataset.map(transform)
dataset1

## Download model

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM
# model_name = "NousResearch/Meta-Llama-3-8B"
model_name = "NousResearch/Meta-Llama-3.1-8B-Instruct"
model_name = "NousResearch/Meta-Llama-3.1-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/50.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

ValueError: `rope_scaling` must be a dictionary with two fields, `type` and `factor`, got {'factor': 8.0, 'low_freq_factor': 1.0, 'high_freq_factor': 4.0, 'original_max_position_embeddings': 8192, 'rope_type': 'llama3'}

# HelpSteer2

https://github.com/jondurbin/bagel/blob/3c7d2410a5a5ad2fd31b63529ef541135feefce4/bagel/data_sources/helpsteer.py#L4

In [1]:
from datasets import load_dataset
# dataset = load_dataset('nvidia/HelpSteer2')
dataset = load_dataset('Atsunori/HelpSteer2-DPO')
dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen_response', 'rejected_response'],
        num_rows: 7221
    })
    validation: Dataset({
        features: ['prompt', 'chosen_response', 'rejected_response'],
        num_rows: 373
    })
})

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
model_name = "NousResearch/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
def proc(row):
    s = row['prompt'] + ' ' + row['chosen_response']
    return tokenizer(s, return_tensors='pt')
d = dataset.map(proc)
d

Map:   0%|          | 0/7221 [00:00<?, ? examples/s]

Map:   0%|          | 0/373 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen_response', 'rejected_response', 'input_ids', 'attention_mask'],
        num_rows: 7221
    })
    validation: Dataset({
        features: ['prompt', 'chosen_response', 'rejected_response', 'input_ids', 'attention_mask'],
        num_rows: 373
    })
})

In [22]:
import pandas as pd
d2 = d.map(lambda x: {'len':len(x['input_ids'][0])})
pd.Series(d2['train']['len']).describe()

count    7221.000000
mean      480.050270
std       294.003984
min         7.000000
25%       267.000000
50%       422.000000
75%       648.000000
max      2611.000000
dtype: float64

In [ ]:
dataset

In [ ]:
import re
from loguru import logger

REFUSALS = [
    re.compile(regexp)
    for regexp in [
        "(against|violate) my programming",
        "openai",
        "language model",
        "large language",
        "as an? (ai|generative language|gpt|bot)",
        "important to note",
        "i do(n't| not) (possess|have|exhibit) (personal|consciousness|subjective)",
        "personal (feelings|thoughts|emotions|desires|experiences|goals|objective|belief)",
        "(can('t| ?not)|w(on't|will not)|unable.?) (\\w+\\s)+(with (that|your)|your \\w+|provide)",
        "my limitations",
        "the limitations of my",
        "my abilities",
        "violates my",
        "i (can('t| ?not)|w(on't|will not)|am (not |un)able.?).{0,30}(you are|you're|your )",
        "please note that",
        "unethical|illegal|dangerous",
        "a text-based",
        "(engag(e|ing)|participat(e|ing)|be involved (in|with)|promot(e|ing)|discuss(ing)?|provid(e|ing))( in)?(\\s*\\w+ that)?(\\s+potentially)? (derogatory|inappropriate|offensive|discriminate|discriminatory|sexist|unacceptable|immoral|unethical|unacceptable|hateful|harmful)",
        "i am commited to",
        "adhere to safety guidelines",
        "maintain user safety",
        "about something else instead",
    ]
]

def has_refusal(response):
    """Check if a response has a refusal - not perfect, but good enough..."""
    for regexp in REFUSALS:
        if m := regexp.search(response, re.I):
            logger.warning(f"Refusal? {m.group()}")
            return True
    return False

.filter(
        lambda item: not has_refusal(item["response"])
    )

In [ ]:
def load_data(known_uids=set([]), **_):
    """HelpSteer dataset for DPO."""
    logger.info("Loading HelpSteer dataset...")
    dataset = load_dataset("nvidia/HelpSteer", split="train").filter(
        lambda item: not has_refusal(item["response"])
    )

    # Use the highest correctness items as the "chosen" responses.
    logger.info("Generating DPO pairs...")
    options = {}
    for item in dataset:
        uid = get_uid(item["prompt"])
        if uid not in options:
            options[uid] = []
        item["score"] = (
            item["helpfulness"]
            + item["correctness"]
            + item["coherence"]
            + item["complexity"]
            + item["verbosity"]
        )
        options[uid].append(item)

    # Select the chosen and rejected items.
    data = []
    for uid, responses in options.items():
        if len(responses) < 2:
            continue
        if uid in known_uids:
            continue
        known_uids.add(uid)
        chosen = None
        chosen_score = 0
        rejected = None
        rejected_score = 0
        for item in responses:
            if item["correctness"] == 4:
                if chosen:
                    if item["score"] > chosen["score"]:
                        rejected = chosen["response"]
                        rejected_score = chosen["score"]
                        chosen = item
                        chosen_score = item["score"]
                        break
                else:
                    chosen = item
                    chosen_score = item["score"]
            else:
                if not rejected:
                    rejected = item["response"]
                    rejected_score = item["score"]
        if chosen and rejected and chosen["response"] != rejected:
            logger.success(f"Found DPO pair: {chosen_score} vs {rejected_score}")
            data.append(
                {
                    "id": uid,
                    "source": "helpsteer",
                    "prompt": item["prompt"],
                    "chosen": chosen["response"],
                    "rejected": rejected,
                    "conversations": None,
                }
            )
    return Dataset.from_list(data)

In [ ]:
dataset['train']['prompt']

## Is norm mean same as mse?

In [4]:
import torch
import torch.nn.functional as F
import torch.nn as nn
# b l t h 
x = torch.randn(2, 3, 4, 5)
y = torch.randn(2, 3, 4 , 5)



In [5]:
F.mse_loss(x, y), torch.norm(y-x, p=2).nanmean(), F.l1_loss(x, y), torch.norm(y-x, p=1).nanmean()

(tensor(2.1555), tensor(16.0827), tensor(1.1736), tensor(140.8272))

In [10]:
F.mse_loss(x, y, reduction='none').nanmean()

tensor(2.1555)

: 

In [ ]:
torch.norm(lora_retain_hidden - orig_retain_hidden, dim=-1, p=2, dtype=torch.float).nanmean()

In [6]:
# the norm is the same as this
# it's not mse, it's sum squared error
e=y-x
((e**2).sum(-1)**(1/2)).mean()

tensor(3.1159)

In [7]:
# these 3 norms act the same
print(torch.norm(y-x, p=2, dim=-1).nanmean())
print(
     torch.linalg.vector_norm(y-x, ord=2, dim=-1).nanmean()
)
torch.linalg.norm(y-x, ord=2, dim=-1).nanmean()

tensor(3.1159)
tensor(3.1159)


tensor(3.1159)

# Try losses

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import torch
import numpy as np
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import DPOTrainer
from trl import DPOConfig, DPOTrainer
import gc
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Any, Callable, Dict, List, Literal, Optional, Tuple, Union
from einops import rearrange

from reprpo.helpers.adapters import set_adapter

In [3]:
# FIXME: we are meant to SFT first, so that the preferences are in sample but 1) if this works it might not be needed, and 2) this can be added later, if it works
# for now we will use the instruct model, and try something it wasn't meant to do but it in sample 
model_name = "NousResearch/Meta-Llama-3-8B-Instruct"

## Big adapter
peft_config = LoraConfig(
    lora_alpha=16, 
    r=16,
    lora_dropout=0.05,
    use_rslora=True,
    # use_dora=True,
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)
from reprpo.models.load import load_model, print_trainable_parameters
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model

model, tokenizer = load_model(model_name, )
# from trl.trainer.utils import peft_module_casting_to_bf16
# peft_module_casting_to_bf16(model)
adapter_name='ReprPO'
model = prepare_model_for_kbit_training(model, {'use_gradient_checkpointing': True})
model = get_peft_model(model, peft_config, adapter_name=adapter_name)
print_trainable_parameters(model)
model

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

trainable params: 41943040 || all params: 4582543360 || trainable%: 0.9152786281546499


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaFlashAttention2(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (ReprPO): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (ReprPO): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (ReprPO): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_layer): Linear4

In [4]:
num_samples = 3

In [5]:
from datasets import load_dataset

def sample(dataset, N):
    return (dataset
            .shuffle(42)
            .select(range(
            min(len(dataset),
                N)))
    )

dataset = load_dataset('Atsunori/HelpSteer2-DPO') # TODO redo exp with
dataset['train'] = sample(dataset['train'], num_samples)
dataset2 = dataset.rename_column('chosen_response', 'chosen').rename_column('rejected_response', 'rejected')

In [6]:
from reprpo.trainer import collect_hs, ReprPOConfig, ReprPOTrainer

In [7]:
training_args = ReprPOConfig('./output-dir/1',
per_device_train_batch_size=2,
                                 gradient_checkpointing=True,
    bf16=True,
    tf32=True,
       max_prompt_length=128,
    max_length=256,
                             )
reprpo_trainer = ReprPOTrainer(
    model=model,
    ref_model=None,
    args=training_args,
    beta=training_args.beta,
    train_dataset=dataset2["train"],
    # eval_dataset=dataset2["test"],
    tokenizer=tokenizer,
)

/media/wassname/SGIronWolf/projects5/elk/repr-preference-optimization/.venv/lib/python3.9/site-packages/trl/trainer/dpo_trainer.py:442: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


In [8]:
dl = reprpo_trainer.get_train_dataloader()
b = next(iter(dl))
b['chosen_input_ids'].shape

torch.Size([3, 256])

In [9]:
# with model.disable_adapter():
with torch.no_grad():
    with reprpo_trainer.null_ref_context():
        (
            reference_chosen_logps,
            reference_rejected_logps,
            _,
            _,
            _,
            reference_chosen_hs,
            _,
        ) = reprpo_trainer.concatenated_forward(reprpo_trainer.model, b)
reference_chosen_hs = reference_chosen_hs.detach()
reference_chosen_logps = reference_chosen_logps.detach()
reference_rejected_logps = reference_rejected_logps.detach()

model.train()
(
    policy_chosen_logps,
    policy_rejected_logps,
    policy_chosen_logits,
    policy_rejected_logits,
    policy_chosen_logps_avg,
    policy_chosen_hs,
    policy_rejected_hs,
) = reprpo_trainer.concatenated_forward(model, b)

The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.


last_N_tok  ['Question', ' you', ' you', "'s", ' the']


ValueError: too many values to unpack (expected 7)

In [ ]:
# loss 1
F.triplet_margin_with_distance_loss(anchor=reference_chosen_hs, positive=policy_chosen_hs, negative=policy_rejected_hs)

F.triplet_margin_with_distance_loss(anchor=reference_chosen_hs, positive=policy_chosen_hs, negative=policy_rejected_hs)

In [ ]:
# reprpo_trainer.concatenated_forward(model, b)

In [ ]:
# model.eval()
# with torch.no_grad():
#     o = model(b)
# o

### Compare various ways of viewing the hidden states!

In [10]:
policy_chosen_hs
policy_rejected_hs

NameError: name 'policy_chosen_hs' is not defined